# AWS Rekognition API Label Detection Example
Michael Grube | June 6, 2019

### Purpose:

To demonstrate how to run images through the AWS Rekognition API to do Label Detection. The resulting csv has a row for each label with greater than 50% confidence.

### Repo used to build out notebook

https://github.com/norawebbwilliams/image_autotaggers

### Modules

In [10]:
import csv
import boto3 
import pickle
import os

### Paths

In [11]:
########### Paths
### MUST ADJUST HERE (1/2)
# Path to where your want to save the resulting labels
rekog_results_dir = 'C:/Users/michael.grube/Documents/GitHub/Rekognition-Example/Data/'
# e.g.:
#rekog_results_dir = 'C:/Users/Nora/Desktop/auto_tagger_example/results/'

# Path to where your images are
rekog_images_dir = 'C:/Users/michael.grube/Documents/GitHub/Rekognition-Example/Images_Label/'
# e.g.:
#rekog_images_dir = 'C:/Users/Nora/Desktop/auto_tagger_example/data/'

### Connect to Rekognition API

In [12]:
# Read in your personal keys
# You can hard code your access key ID and secret key ID into the script, 
# but this is not recommended

personal_access_key = "your_personal_access_key"
secret_access_key = "your_secret_access_key"

# Instead we recommend storing your keys securely in a csv or text file.
# For example, if you have saved your keys in a csv:

credentials = []

### MUST ADJUST HERE (2/2)
with open('C:/Users/michael.grube/Documents/AWS Access Keys/accessKeysAdministrator.csv', newline='') as csvfile:

# e.g.:
#with open('C:/Users/Nora/Desktop/auto_tagger_example/keys/AWS_personal_nora_admin_credentials.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        credentials.append(row)

personal_access_key = credentials[0]['Access key ID']
secret_access_key = credentials[0]['Secret access key']

# Initialize the boto client to access the Rekogniton api
client=boto3.client('rekognition','us-east-1', # or choose the best region for your work, 
                                               # e.g. the location of your S3 bucket if using that method to store images
                    aws_access_key_id = personal_access_key, 
                    aws_secret_access_key = secret_access_key) 

### Create a list of images to run through the API

In [13]:
# Make a list of all the images in the rekog_data_dir you created
local_images = os.listdir(rekog_images_dir)

### Run each image through the API and store the results

In [36]:
##### Detect label
## 
holder_labels = []

for imageFile in local_images:
    with open(rekog_images_dir + imageFile, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
    
    print('Detected Labels for ' + imageFile)
    
    ## If no label detected, still save the info:
    if len(response['Labels']) == 0:
        print ("No Label Detected")
        temp_dict = {}
        temp_dict["image_id"] = imageFile
        temp_dict["full_detect_labels_response"] = response
        temp_dict["label_num"] = ''
        temp_dict["label_str"] = ''
        temp_dict["label_conf"] = ''
        temp_dict["label_parents"] = ''
        holder_labels.append(temp_dict)   
    
    else:
        
        label_counter = 1
        
        for label in response['Labels']:
            print (label['Name'] + ' : ' + str(label['Confidence']) + '%')
            print (label['Parents'])
            temp_dict = {}
            temp_dict["image_id"] = imageFile
            temp_dict["full_detect_labels_response"] = response
            temp_dict["label_num"] = label_counter
            temp_dict["label_str"] = label['Name']
            temp_dict["label_conf"] = label['Confidence']
            temp_dict["label_parents"] = label['Parents']
            label_counter +=1 # update for the next label
            holder_labels.append(temp_dict)
          
len(holder_labels)


Detected Labels for cherries.jpeg
Plant : 99.62244415283203%
[]
Fruit : 97.097412109375%
[{'Name': 'Plant'}, {'Name': 'Food'}]
Food : 97.097412109375%
[]
Cherry : 87.27527618408203%
[{'Name': 'Fruit'}, {'Name': 'Plant'}, {'Name': 'Food'}]
Detected Labels for eclipse.jpg
Astronomy : 99.05462646484375%
[]
Eclipse : 99.05462646484375%
[{'Name': 'Astronomy'}]
Night : 98.57241821289062%
[{'Name': 'Nature'}, {'Name': 'Outdoors'}]
Nature : 98.57241821289062%
[]
Space : 98.57241821289062%
[{'Name': 'Astronomy'}]
Outer Space : 98.57241821289062%
[{'Name': 'Astronomy'}]
Universe : 98.57241821289062%
[{'Name': 'Astronomy'}]
Outdoors : 98.57241821289062%
[]
Moon : 98.57241821289062%
[{'Name': 'Outer Space'}, {'Name': 'Astronomy'}, {'Name': 'Night'}, {'Name': 'Nature'}, {'Name': 'Outdoors'}]
Lunar Eclipse : 95.44381713867188%
[{'Name': 'Outer Space'}, {'Name': 'Moon'}, {'Name': 'Astronomy'}, {'Name': 'Eclipse'}, {'Name': 'Night'}, {'Name': 'Nature'}, {'Name': 'Outdoors'}]
Detected Labels for paintb

29

### Write out the results to a csv

In [37]:
# Write out the results to a csv
with open(rekog_results_dir + 'awsrekognition_detect_label.csv', 'w', newline = '') as csvfile:
    fieldnames = ['image_id', 'full_detect_labels_response',
                  'label_num', 'label_str',
                  'label_conf', 'label_parents'
                  ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader() 
    for entry in holder_labels:
        writer.writerow(entry)
        
print("Success")

Success
